In [ ]:
# I'll read in all of the frame metadata, and just save the the indexes as new df's to the right location

# dude I think that I could just flatten all the video files and move them

# I'd do a smilar check to see if I need to split

In [ ]:
import os
import pandas as pd

deskCamInfo = ["abhik", "deskCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
bathroomCamInfo = ["abhik", "bathroomCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]

deskCamLocation = "/home/chowder/Documents/oldworkingData/homeVideo/deskCam/frameMetaData/"
for i, fn in enumerate(sorted(os.listdir(deskCamLocation))):
    if i == 0:
        deskCamDf = pd.read_parquet(deskCamLocation + fn)
    else:
        deskCamDf = pd.concat([deskCamDf, pd.read_parquet(deskCamLocation + fn)])

deskCamDf = deskCamDf.drop(columns=deskCamDf.columns).sort_index()

# now we could save the deskCamDf to the right spot
# but first we're going to flatten the deskcam directory
# find deskCam/ -type f -exec cp -n {} dc/ \;
deskCamVidLocation = "/home/chowder/Documents/videoData/dc/"

In [29]:
bathroomCamLocation = "/home/chowder/Documents/oldworkingData/homeVideo/bathroomCam/frameMetaData/"
for i, fn in enumerate(sorted(os.listdir(bathroomCamLocation))):
    if i == 0:
        bathroomCamDf = pd.read_parquet(bathroomCamLocation + fn)
    else:
        bathroomCamDf = pd.concat([bathroomCamDf, pd.read_parquet(bathroomCamLocation + fn)])

bathroomCamDf = bathroomCamDf.drop(columns=bathroomCamDf.columns).sort_index()
#mkdir bc
# find bathroomCam/ -type f -exec cp -n {} bc/ \;

bathroomCamVidLocation = "/home/chowder/Documents/videoData/bc/"

In [3]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
import os
import sys
import subprocess
import cv2
import shutil

In [25]:
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()

In [18]:
def geBulkLocation(time, responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource):
    dataFolderName = "_".join([responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource]) + "/"
    dataFolderName += time.strftime("%Y") + "/" + time.strftime("%m-%d") + "/"
    return repoPath + "bulkData/" + dataFolderName

def fnString_to_dt(fn_string):
    fn_string = fn_string[:17] + ',' + fn_string[18:]
    return datetime.fromisoformat(fn_string.replace(",", "."))

def dt_to_fnString(dt):
    return dt.astimezone(ZoneInfo("UTC")).strftime('%Y-%m-%dT%H%M%S,%f%z')

def splitVidopencv(input_file_location, boundary_indexes, output_paths):
    cap = cv2.VideoCapture(input_file_location)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    
    for i, (start_frame, end_frame) in enumerate(boundary_indexes):
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out = cv2.VideoWriter(output_paths[i], fourcc, fps, (width, height))

        for frame_idx in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        print(f"Saved: {output_paths[i]}")

    cap.release()

In [27]:
# honestly we could make a function that takes in a directory containing a bunch of mp4s and splits them if needed
def copyAndSplitMp4(folderPath, timeDF, camDetails):
    vidFileNames = sorted(os.listdir(folderPath))
    for fileName in vidFileNames:
        startTime = fnString_to_dt(fileName.split("_")[-2]).astimezone(ZoneInfo("UTC"))
        endTime = fnString_to_dt(fileName.split("_")[-1][:-4]).astimezone(ZoneInfo("UTC"))
        
        if startTime.day == endTime.day:
            newFileName = dt_to_fnString(startTime) + "_" + dt_to_fnString(endTime) + ".mp4"
            source = folderPath + fileName
            destination = geBulkLocation(startTime, *camDetails)
            if not os.path.exists(destination):
                print("made directory " + destination)
                os.makedirs(destination)
            shutil.copy2(source, destination  + newFileName)
            continue
        
        # the file contains more than one day of data
        frameTimes = timeDF[startTime:endTime].copy()
        frameTimes['date'] = frameTimes.index.normalize()
        boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]), include_groups=False)
        index_boundaries = [(frameTimes.index.get_loc(start), frameTimes.index.get_loc(end))
                                for day, (start, end) in boundaries.items()]
        
        output_paths = [geBulkLocation(x[0], *camDetails) + 
                        dt_to_fnString(x[0]) + "_" + dt_to_fnString(x[1]) + ".mp4" 
                        for x in boundaries]

        splitVidopencv(folderPath + fileName  + ".mp4", index_boundaries, output_paths)
    

In [31]:
copyAndSplitMp4(deskCamVidLocation, deskCamDf, deskCamInfo)

made directory /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-11/
Saved: /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-11/2024-11-11T233021,700087+0000_2024-11-12T001151,100085+0000.mp4
made directory /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-12/
Saved: /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-12/2024-11-12T235701,100096+0000_2024-11-13T000001,000089+0000.mp4
made directory /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-13/
Saved: /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-13/2024-11-13T231630,000158+0000_2024-11-14T043836,400084+0000.mp4
made directory /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-14/
made directory /home/chowder/Documents/bulkData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-15/
Sav

In [32]:
copyAndSplitMp4(bathroomCamVidLocation, bathroomCamDf, bathroomCamInfo)

made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-15/
Saved: /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-15/2024-11-15T235745,900079+0000_2024-11-16T000045,800127+0000.mp4
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-16/
Saved: /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-16/2024-11-16T184245,000164+0000_2024-11-17T021230,000133+0000.mp4
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-17/
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-18/
Saved: /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/11-18/2024-11-18T211200,000188+0000_2024-11-19T032301,700140+0000.mp4
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-w

In [39]:
sys.path.append(repoPath + "dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import writeWorkingTSDf, readWorkingTSDF, fnString_to_dt, dt_to_fnString

In [40]:
writeWorkingTSDf(*bathroomCamInfo, bathroomCamDf)
writeWorkingTSDf(*deskCamInfo, deskCamDf)

hashes match for 2024-11-15T011915,001407+0000_2024-11-16T010456,900094+0000_56d9bfa8_.parquet.gzip
the hashes don't match for start time 2024-11-16 01:04:56.900094+00:00 to end time 2024-11-22 06:19:35.700134+00:00
reading and combining to see if there's any new data
hashes match now
hashes match for 2024-11-22T061935,700134+0000_2024-12-11T063645,000290+0000_f490d032_.parquet.gzip
the hashes don't match for start time 2024-12-11 06:36:45.000290+00:00 to end time 2024-12-28 19:27:53.700100+00:00
reading and combining to see if there's any new data
hashes match now
nothing to add
the hashes don't match for start time 1677-09-21 00:12:43.145224193+00:00 to end time 2024-11-12 06:36:30.400087+00:00
reading and combining to see if there's any new data
hashes match now
hashes match for 2024-11-12T063630,400087+0000_2024-11-13T130813,700087+0000_fc771c44_.parquet.gzip
the hashes don't match for start time 2024-11-13 13:08:13.700087+00:00 to end time 2024-11-15 05:09:30.300083+00:00
reading 

In [38]:
import importlib
importlib.reload(rwWorkingTSDf)

<module 'rwWorkingTSDf' from '/home/chowder/Documents/dataImport/rwWorkingTSDf.py'>